<a href="https://colab.research.google.com/github/madeyemocki/xai_fnd_data/blob/main/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
import requests
import io
from urllib.request import urlopen
import json
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.io.json import json_normalize
from pprint import pprint
from scipy.stats import shapiro


In [2]:
# installing mne for the BH correction
!pip install mne
import mne

!pip install scikit_posthocs
import scikit_posthocs as sp

!pip install statannotations

!pip install pingouin
import pingouin as pg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.12.2
    Uninstalling seaborn-0.12.2:
      Successfully uninstalled seaborn-0.12.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=259f011135ca235edc75ab668b8f75dc8332b111c6864cdb086c8c89786a6ddf
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [129]:
#Ausgangs Dataframes 
df_v1_laymen = pd.read_csv('data/v1_cleaned_laymen.csv')
df_v2_laymen = pd.read_csv('data/v2_cleaned_laymen.csv')
df_v3_laymen = pd.read_csv('data/v3_cleaned_laymen.csv')

# merge the dataframes
df_all_versions = pd.concat([df_v1_laymen, df_v2_laymen, df_v3_laymen], ignore_index=True)

In [ ]:
#check
columns = list(df_v3_laymen.columns)
pprint(columns)

# Funktionen

In [ ]:
def find_highest_alpha(data):
    alpha_values = {}  # Dictionary zur Speicherung der Alpha-Werte

    # Iteriere über jedes Item und berechne den Alpha-Wert, indem das Item weggelassen wird
    for item in data.columns:
        items_without_current = data.drop(item, axis=1)  # DataFrame ohne das aktuelle Item
        alpha = pg.cronbach_alpha(data=items_without_current) # Alpha-Wert berechnen
        alpha_values[item] = alpha  # Alpha-Wert in das Dictionary speichern

    # Finde das Item, das entfernt werden sollte, um den höchsten Alpha-Wert zu erzielen
    max_alpha_item = max(alpha_values, key=alpha_values.get)
    max_alpha = alpha_values[max_alpha_item]

    return max_alpha_item, max_alpha

In [125]:
def calculate_alpha(df):
    return pg.cronbach_alpha(data=df) 



In [73]:
def calculate_stats(df):
    results = []
    for column in df.columns:
        shapiro_test = shapiro(df[column])
        mean = df[column].mean()
        median = df[column].median()
        results.append((column, shapiro_test.statistic, shapiro_test.pvalue, mean, median))

    # Create a DataFrame with the results
    df_results = pd.DataFrame(results, columns=['Column', 'Shapiro-Wilk', 'p-value', 'Mean', 'Median'])
    return df_results

#df_results
# Usage example


,Column,Shapiro-Wilk,p-value,Mean,Median
0,information-literacy.access-relevant-literature,0.745027,2.333691e-14,5.943262,6.0
1,information-literacy.check-reliability,0.781103,3.096309e-13,5.886525,6.0
2,information-literacy.differentiate-right-and-fake,0.859654,2.974758e-10,5.560284,6.0
3,information-literacy.efficiently-use-subscribe...,0.819892,7.120578e-12,5.758865,6.0
4,information-literacy.have-it-skills,0.886317,5.464746e-09,5.056738,5.0
5,information-literacy.identify-authentic-sources,0.865856,5.658727e-10,5.652482,6.0
6,information-literacy.identify-misinformation,0.870511,9.286336e-10,5.304965,5.0
7,information-literacy.plagiarism-and-publicatio...,0.800596,1.420148e-12,5.907801,6.0
8,information-literacy.research-level-skilss,0.839627,4.202284e-11,5.659574,6.0


# Demographics
in prozent?

In [6]:
# get columns which start with "demographics"
demographics_cols = [col for col in df_all_versions.columns if col.startswith('demographics')]
demographics_cols
# country and nationality are not relevant, since the study was conducted in the US

['demographics.age',
 'demographics.country',
 'demographics.education',
 'demographics.employment',
 'demographics.gender',
 'demographics.income',
 'demographics.nationality']

In [7]:
for col in demographics_cols:
    value_counts = df_all_versions[col].value_counts()
    print(f"Spalte: {col}")
    print(value_counts)
    print("\n")

# Usa, USA... zusammenfassen?

Spalte: demographics.age
30-39          153
21-29          100
40-49           88
50-59           25
18-20           17
60-or-older      7
Name: demographics.age, dtype: int64


Spalte: demographics.country
United States                            158
USA                                       81
United States of America                  36
United States                             22
US                                        13
united states                             12
Usa                                       10
usa                                        9
USA                                        8
United states                              6
America                                    3
U.S.A                                      3
united states                              3
U.S.                                       3
United States Of America                   2
united states of america                   2
United States of America                   2
United states               

# Information Literacy
for each version?

In [13]:
information_literacy_cols = [col for col in df_all_versions.columns if col.startswith('information-literacy')]
information_literacy_cols

# Extrahiere die gemeinsamen Spalten
common_cols_v1 = df_v1_laymen.columns.intersection(information_literacy_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v1_information_literacy_laymen = df_v1_laymen[common_cols_v1].copy()

# Extrahiere die gemeinsamen Spalten
common_cols_v2 = df_v2_laymen.columns.intersection(information_literacy_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v2_information_literacy_laymen = df_v2_laymen[common_cols_v2].copy()

# Extrahiere die gemeinsamen Spalten
common_cols_v3 = df_v3_laymen.columns.intersection(information_literacy_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v3_information_literacy_laymen = df_v3_laymen[common_cols_v3].copy()


In [14]:
#Chronbachs Alpha reliability of information literacy
info_lit = df_v1_laymen[['information-literacy.access-relevant-literature',
       'information-literacy.check-reliability',
       'information-literacy.differentiate-right-and-fake',
       'information-literacy.efficiently-use-subscribed-and-openaccess',
       'information-literacy.have-it-skills',
       'information-literacy.identify-authentic-sources',
       'information-literacy.identify-misinformation',
       'information-literacy.plagiarism-and-publication-ethics',
       'information-literacy.research-level-skilss']]
pg.cronbach_alpha(data=info_lit) 
#first position is cronbachs alpha, second is confidence interval 95% (0.9 excellent, 0.8 good, 0.7 accaptable)

(0.9269078654422946, array([0.907, 0.944]))

In [15]:
#Chronbachs Alpha reliability of prot behav
info_lit = df_v2_laymen[['information-literacy.access-relevant-literature',
       'information-literacy.check-reliability',
       'information-literacy.differentiate-right-and-fake',
       'information-literacy.efficiently-use-subscribed-and-openaccess',
       'information-literacy.have-it-skills',
       'information-literacy.identify-authentic-sources',
       'information-literacy.identify-misinformation',
       'information-literacy.plagiarism-and-publication-ethics',
       'information-literacy.research-level-skilss']]
pg.cronbach_alpha(data=info_lit) 

(0.8761898427566921, array([0.838, 0.909]))

In [16]:
#Chronbachs Alpha reliability of prot behav
info_lit = df_v3_laymen[['information-literacy.access-relevant-literature',
       'information-literacy.check-reliability',
       'information-literacy.differentiate-right-and-fake',
       'information-literacy.efficiently-use-subscribed-and-openaccess',
       'information-literacy.have-it-skills',
       'information-literacy.identify-authentic-sources',
       'information-literacy.identify-misinformation',
       'information-literacy.plagiarism-and-publication-ethics',
       'information-literacy.research-level-skilss']]
pg.cronbach_alpha(data=info_lit) 

(0.9110326051832224, array([0.887, 0.932]))

In [17]:
#Normalverteilung

# Calculate Shapiro-Wilk, mean, and median for each column
results = []
for column in df_v1_information_literacy_laymen.columns:
    shapiro_test = shapiro(df_v1_information_literacy_laymen[column])
    mean = df_v1_information_literacy_laymen[column].mean()
    median = df_v1_information_literacy_laymen[column].median()
    results.append((column, shapiro_test.statistic, shapiro_test.pvalue, mean, median))

# Create a DataFrame with the results
df_results = pd.DataFrame(results, columns=['Column', 'Shapiro-Wilk', 'p-value', 'Mean', 'Median'])

# Display the DataFrame
df_results
# If the chosen alpha level is 0.05 and the p-value is less than 0.05, then the null hypothesis that the data are normally distributed is rejected
# p-value bei Shapiro wenn unter 0.05 dann keine Normalverteilung dann nicht parametrische tests
# the data in these columns do not follow a normal distribution. 
# Information Literacy ins zusammenhang mit fp tn erkennung bringen, was wenn bei einem parametrische tests gemacht werden sollen und beim andern parmetrische?
# Wann part wo ich nochmal p value messe?



,Column,Shapiro-Wilk,p-value,Mean,Median
0,information-literacy.access-relevant-literature,0.799605,1.167457e-12,5.964789,6.0
1,information-literacy.check-reliability,0.860818,3.029834e-10,5.830986,6.0
2,information-literacy.differentiate-right-and-fake,0.867269,5.949396e-10,5.739437,6.0
3,information-literacy.efficiently-use-subscribe...,0.846471,7.244923e-11,5.676056,6.0
4,information-literacy.have-it-skills,0.903314,4.040287e-08,5.042254,5.0
5,information-literacy.identify-authentic-sources,0.840797,4.215798e-11,5.661972,6.0
6,information-literacy.identify-misinformation,0.870772,8.660047e-10,5.380282,5.5
7,information-literacy.plagiarism-and-publicatio...,0.802334,1.456426e-12,5.887324,6.0
8,information-literacy.research-level-skilss,0.843617,5.508796e-11,5.718310,6.0


In [18]:
#Normalverteilung

# Calculate Shapiro-Wilk, mean, and median for each column
results = []
for column in df_v2_information_literacy_laymen.columns:
    shapiro_test = shapiro(df_v2_information_literacy_laymen[column])
    mean = df_v2_information_literacy_laymen[column].mean()
    median = df_v2_information_literacy_laymen[column].median()
    results.append((column, shapiro_test.statistic, shapiro_test.pvalue, mean, median))

# Create a DataFrame with the results
df_results = pd.DataFrame(results, columns=['Column', 'Shapiro-Wilk', 'p-value', 'Mean', 'Median'])

# Display the DataFrame
df_results
# If the chosen alpha level is 0.05 and the p-value is less than 0.05, then the null hypothesis that the data are normally distributed is rejected
# p-value bei Shapiro wenn unter 0.05 dann keine Normalverteilung dann nicht parametrische tests
# the data in these columns do not follow a normal distribution. 
# Information Literacy ins zusammenhang mit fp tn erkennung bringen, was wenn bei einem parametrische tests gemacht werden sollen und beim andern parmetrische?
# Wann part wo ich nochmal p value messe?



,Column,Shapiro-Wilk,p-value,Mean,Median
0,information-literacy.access-relevant-literature,0.750139,3.318344e-12,6.074766,6.0
1,information-literacy.check-reliability,0.830571,9.847583e-10,6.084112,6.0
2,information-literacy.differentiate-right-and-fake,0.855474,8.195029e-09,5.766355,6.0
3,information-literacy.efficiently-use-subscribe...,0.870286,3.229915e-08,5.747664,6.0
4,information-literacy.have-it-skills,0.904524,1.160630e-06,4.953271,5.0
5,information-literacy.identify-authentic-sources,0.865840,2.119202e-08,5.813084,6.0
6,information-literacy.identify-misinformation,0.867799,2.548819e-08,5.514019,6.0
7,information-literacy.plagiarism-and-publicatio...,0.791589,5.187702e-11,5.925234,6.0
8,information-literacy.research-level-skilss,0.825391,6.501285e-10,5.869159,6.0


In [19]:
#Normalverteilung

# Calculate Shapiro-Wilk, mean, and median for each column
results = []
for column in df_v3_information_literacy_laymen.columns:
    shapiro_test = shapiro(df_v3_information_literacy_laymen[column])
    mean = df_v3_information_literacy_laymen[column].mean()
    median = df_v3_information_literacy_laymen[column].median()
    results.append((column, shapiro_test.statistic, shapiro_test.pvalue, mean, median))

# Create a DataFrame with the results
df_results = pd.DataFrame(results, columns=['Column', 'Shapiro-Wilk', 'p-value', 'Mean', 'Median'])

# Display the DataFrame
df_results
# If the chosen alpha level is 0.05 and the p-value is less than 0.05, then the null hypothesis that the data are normally distributed is rejected
# p-value bei Shapiro wenn unter 0.05 dann keine Normalverteilung dann nicht parametrische tests
# the data in these columns do not follow a normal distribution. 
# Information Literacy ins zusammenhang mit fp tn erkennung bringen, was wenn bei einem parametrische tests gemacht werden sollen und beim andern parmetrische?
# Wann part wo ich nochmal p value messe?


,Column,Shapiro-Wilk,p-value,Mean,Median
0,information-literacy.access-relevant-literature,0.745027,2.333691e-14,5.943262,6.0
1,information-literacy.check-reliability,0.781103,3.096309e-13,5.886525,6.0
2,information-literacy.differentiate-right-and-fake,0.859654,2.974758e-10,5.560284,6.0
3,information-literacy.efficiently-use-subscribe...,0.819892,7.120578e-12,5.758865,6.0
4,information-literacy.have-it-skills,0.886317,5.464746e-09,5.056738,5.0
5,information-literacy.identify-authentic-sources,0.865856,5.658727e-10,5.652482,6.0
6,information-literacy.identify-misinformation,0.870511,9.286336e-10,5.304965,5.0
7,information-literacy.plagiarism-and-publicatio...,0.800596,1.420148e-12,5.907801,6.0
8,information-literacy.research-level-skilss,0.839627,4.202284e-11,5.659574,6.0


# Information seeking behavior

In [ ]:
columns = list(df_all_versions.columns)
pprint(columns)

In [ ]:
information_seeking_behavior_cols = [col for col in df_all_versions.columns if col.startswith('information-seeking')]
information_seeking_behavior_cols

# Extrahiere die gemeinsamen Spalten
common_cols_v1 = df_v1.columns.intersection(information_seeking_behavior_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v1_information_seeking_behavior = df_v1[common_cols_v1].copy()

# Extrahiere die gemeinsamen Spalten
common_cols_v2 = df_v2.columns.intersection(information_seeking_behavior_cols)

# Erstelle ein neues DataFrame mit den gemeinsamen Spalten
df_v2_information_seeking_behavior = df_v2[common_cols_v2].copy()

pprint(df_v2_information_seeking_behavior)



In [48]:

# Access
# Spaltennamen definieren
column_names_access = ['information-seeking.access.google', 
                       'information-seeking.access.library',
                       'information-seeking.access.personal-purchase',
                       'information-seeking.access.websites',
                       'information-seeking.accessibility-of-information-about-transparency',
                       'information-seeking.comprehensive-information']

# DataFrame mit den Spalten erstellen
df_v1_isb_access = df_v1_information_seeking_behavior[column_names_access].copy()

#Sources
# Spaltennamen definieren
column_names_src = [
    'information-seeking.sources.blogs-websites',
    'information-seeking.sources.facebook',
    'information-seeking.sources.instagram',
    'information-seeking.sources.interview',
    'information-seeking.sources.newspaper',
    'information-seeking.sources.online-newspaper',
    'information-seeking.sources.personal-social-networks',
    'information-seeking.sources.twitter',
    'information-seeking.most-used-sources.0',
    'information-seeking.most-used-sources.1',
    'information-seeking.most-used-sources.2',
    'information-seeking.top-source-seriousness',
    'information-seeking.truthfulness.facebook',
    'information-seeking.truthfulness.instagram',
    'information-seeking.truthfulness.newspaper',
    'information-seeking.truthfulness.online-newspaper',
    'information-seeking.truthfulness.twitter'
]

# DataFrame mit den Spalten erstellen
df_v1_isb_sources = df_v1_information_seeking_behavior[column_names_src].copy()

# Purpose
# Spaltennamen definieren
column_names_purp = ['information-seeking.purpose.general-awareness',
 'information-seeking.purpose.keep-up-to-date',
 'information-seeking.purpose.preparing-research']

# DataFrame mit den Spalten erstellen
df_v1_isb_purpose = df_v1_information_seeking_behavior[column_names_purp].copy()


df_v1_isb_sources
df_v1_isb_access
df_v1_isb_purpose

,information-seeking.purpose.general-awareness,information-seeking.purpose.keep-up-to-date,information-seeking.purpose.preparing-research
0,6,6,3
1,7,7,7
2,7,7,7
3,4,4,5
4,6,6,4
...,...,...,...
137,3,3,4
138,7,7,4
139,6,5,5
140,6,6,4


In [ ]:
# Access
prot_beh = df_v1_isb_access [['information-seeking.accessibility-of-information-about-transparency', 'information-seeking.comprehensive-information']]
pg.cronbach_alpha(data=prot_beh) 

#wie geh ich hier vor??

(0.4454339277168602, array([0.227, 0.602]))

In [ ]:
# DataFrame mit der relevanten Spalte erstellen
df_v1_isb_access = df_v1_information_seeking_behavior [['information-seeking.comprehensive-information']]

# Cronbach's Alpha berechnen
alpha = df_v1_isb_access['information-seeking.comprehensive-information'].var()

print("Cronbach's Alpha:", alpha)

Cronbach's Alpha: 0.959994006592748


In [41]:
# DataFrame mit den relevanten Spalten erstellen
df_v1_isb_access_2 = df_v1_information_seeking_behavior[['information-seeking.access.google', 'information-seeking.access.library', 'information-seeking.access.personal-purchase', 'information-seeking.access.websites']]

# Korrelationen berechnen
correlations = df_v1_isb_access_2.corr()

# Item-Total-Korrelationen für jede Spalte anzeigen
for column in df_v1_isb_access_2.columns:
    item_total_corr = correlations[column].drop(column).mean()
    print("Item-Total-Korrelation for", column)
    print("Item-Total-Korrelation:", item_total_corr)
    print("")


Item-Total-Korrelation for information-seeking.access.google
Item-Total-Korrelation: 0.2209861812796725

Item-Total-Korrelation for information-seeking.access.library
Item-Total-Korrelation: 0.13822184629049886

Item-Total-Korrelation for information-seeking.access.personal-purchase
Item-Total-Korrelation: 0.1818651672050595

Item-Total-Korrelation for information-seeking.access.websites
Item-Total-Korrelation: 0.2804994311292584



# General Toughts about AI

In [ ]:
#check
#columns = list(df_v1_laymen.columns)
#pprint(columns)

In [94]:
expectations_columns = [col for col in df_v1_laymen.columns if col.startswith('expectations')]

# Erstelle ein neues DataFrame mit den extrahierten Spalten
df_v1_expectations_before_laymen = df_v1_laymen[expectations_columns].copy()
df_v1_expectations_before_laymen

#sceptical reverse coded .expectations.sceptical-about-ai

# Reverse Coding der entsprechenden Frage
df_v1_expectations_before_laymen['expectations.sceptical-about-ai_reverse'] = 8 - df_v1_expectations_before_laymen['expectations.sceptical-about-ai']

df_v1_expectations_before_laymen.drop(['expectations.sceptical-about-ai'], axis=1, inplace=True)

df_v1_expectations_before_laymen
#chronbachs alpha 
print('Chronbachs Alpha for df_v1_expectations_before_laymen')
pg.cronbach_alpha(data= df_v1_expectations_before_laymen) 


Chronbachs Alpha for df_v1_expectations_before_laymen


(0.8017803642673864, array([0.745, 0.849]))

In [95]:
column_names = [
    'ai-system-evaluation.benefit-of-doubt',
    'ai-system-evaluation.rely-on-ai-system',
    'ai-system-evaluation.sceptical-about-ai-system',
    'ai-system-evaluation.trust-new-tech'
]

# DataFrame mit den Spalten erstellen
df_v1_expectations_after_laymen = df_v1_laymen[column_names].copy()

# Reverse Coding der entsprechenden Frage
df_v1_expectations_after_laymen['ai-system-evaluation.sceptical-about-ai-system_reverse'] = 8 - df_v1_expectations_after_laymen['ai-system-evaluation.sceptical-about-ai-system']

df_v1_expectations_after_laymen.drop(['ai-system-evaluation.sceptical-about-ai-system'], axis=1, inplace=True)
df_v1_expectations_after_laymen

#chronbachs alpha 
print('Chronbachs Alpha for df_v1_expectations_after_laymen')
exp_after = df_v1_expectations_after_laymen
pg.cronbach_alpha(data=df_v1_expectations_after_laymen) 
# da niedriger als 0,7 highest alpha func verwenden

Chronbachs Alpha for df_v1_expectations_after_laymen


(0.6528518961015974, array([0.549, 0.738]))

In [96]:
# Höchsten Cronbach's Alpha-Wert finden
item, alpha = find_highest_alpha(df_v1_expectations_after_laymen)

print("Highest Cronbach's Alpha:", alpha)
print("Item to remove:", item)

# dann auch im before raus?? todo
df_v1_expectations_after_laymen.drop(['ai-system-evaluation.sceptical-about-ai-system_reverse'], axis=1, inplace=True)
# dann auch für before wegen vergleich
df_v1_expectations_before_laymen.drop(['expectations.sceptical-about-ai_reverse'], axis=1, inplace=True)

#chronbach neu:
print('New Chronbachs Alpha for df_v1_expectations_after_laymen')
pg.cronbach_alpha(data=df_v1_expectations_after_laymen) 


Highest Cronbach's Alpha: (0.7937497235547442, array([0.727, 0.846]))
Item to remove: ai-system-evaluation.sceptical-about-ai-system_reverse
New Chronbachs Alpha for df_v1_expectations_after_laymen


(0.7937497235547442, array([0.727, 0.846]))

In [97]:
print('New Chronbachs Alpha for df_v1_expectations_before_laymen')
pg.cronbach_alpha(data=df_v1_expectations_before_laymen) 

New Chronbachs Alpha for df_v1_expectations_before_laymen


(0.8371206904466784, array([0.789, 0.877]))

In [101]:
print('Shapiro Wilk for df_v1_expectations_before_laymen')
calculate_stats(df_v1_expectations_before_laymen)

#Trust in AI raus für overall vergleich???

Shapiro Wilk for df_v1_expectations_before_laymen


,Column,Shapiro-Wilk,p-value,Mean,Median
0,expectations.benefit-of-doubt,0.884213,3.896461e-09,5.161972,5.0
1,expectations.rely-on-ai-for-fake-news-detection,0.931089,2.114667e-06,4.760563,5.0
2,expectations.trust-ai,0.910511,1.048802e-07,4.704225,5.0
3,expectations.trust-new-tech,0.916882,2.536628e-07,4.880282,5.0


In [102]:
print('Shapiro Wilk for df_v1_expectations_after_laymen')

calculate_stats(df_v1_expectations_after_laymen)

Shapiro Wilk for df_v1_expectations_after_laymen


,Column,Shapiro-Wilk,p-value,Mean,Median
0,ai-system-evaluation.benefit-of-doubt,0.888312,6.300658e-09,5.253521,5.0
1,ai-system-evaluation.rely-on-ai-system,0.924295,7.458904e-07,4.591549,5.0
2,ai-system-evaluation.trust-new-tech,0.911751,1.241912e-07,4.894366,5.0


In [103]:
expectations_columns = [col for col in df_v2_laymen.columns if col.startswith('expectations')]

# Erstelle ein neues DataFrame mit den extrahierten Spalten
df_v2_expectations_before_laymen = df_v2_laymen[expectations_columns].copy()
df_v2_expectations_before_laymen

#sceptical reverse coded .expectations.sceptical-about-ai

# Reverse Coding der entsprechenden Frage
df_v2_expectations_before_laymen['expectations.sceptical-about-ai_reverse'] = 8 - df_v2_expectations_before_laymen['expectations.sceptical-about-ai']

df_v2_expectations_before_laymen.drop(['expectations.sceptical-about-ai'], axis=1, inplace=True)

df_v2_expectations_before_laymen
#chronbachs alpha 
print('Chronbachs Alpha for df_v2_expectations_before_laymen')
pg.cronbach_alpha(data= df_v2_expectations_before_laymen) 


Chronbachs Alpha for df_v2_expectations_before_laymen


(0.8388518316604132, array([0.785, 0.883]))

In [104]:
column_names = [
    'ai-system-evaluation.benefit-of-doubt',
    'ai-system-evaluation.rely-on-ai-system',
    'ai-system-evaluation.sceptical-about-ai-system',
    'ai-system-evaluation.trust-new-tech'
]

# DataFrame mit den Spalten erstellen
df_v2_expectations_after_laymen = df_v1_laymen[column_names].copy()

# Reverse Coding der entsprechenden Frage
df_v2_expectations_after_laymen['ai-system-evaluation.sceptical-about-ai-system_reverse'] = 8 - df_v2_expectations_after_laymen['ai-system-evaluation.sceptical-about-ai-system']

df_v2_expectations_after_laymen.drop(['ai-system-evaluation.sceptical-about-ai-system'], axis=1, inplace=True)
df_v2_expectations_after_laymen

#chronbachs alpha 
print('Chronbachs Alpha for df_v2_expectations_after_laymen')
exp_after = df_v2_expectations_after_laymen
pg.cronbach_alpha(data=df_v2_expectations_after_laymen) 
# da niedriger als 0,7 highest alpha func verwenden

Chronbachs Alpha for df_v2_expectations_after_laymen


(0.6528518961015974, array([0.549, 0.738]))

In [105]:
# Höchsten Cronbach's Alpha-Wert finden
item, alpha = find_highest_alpha(df_v2_expectations_after_laymen)

print("Highest Cronbach's Alpha:", alpha)
print("Item to remove:", item)

# dann auch im before raus?? todo
df_v2_expectations_after_laymen.drop(['ai-system-evaluation.sceptical-about-ai-system_reverse'], axis=1, inplace=True)

df_v2_expectations_before_laymen.drop(['expectations.sceptical-about-ai_reverse'], axis=1, inplace=True)

print('New Chronbachs Alpha for df_v2_expectations_after_laymen')
pg.cronbach_alpha(data=df_v2_expectations_after_laymen) 

Highest Cronbach's Alpha: (0.7937497235547442, array([0.727, 0.846]))
Item to remove: ai-system-evaluation.sceptical-about-ai-system_reverse
New Chronbachs Alpha for df_v2_expectations_after_laymen


(0.7937497235547442, array([0.727, 0.846]))

In [106]:
print('New Chronbachs Alpha for df_v2_expectations_before_laymen')
pg.cronbach_alpha(data=df_v2_expectations_before_laymen) 

New Chronbachs Alpha for df_v2_expectations_before_laymen


(0.7982522450073715, array([0.728, 0.854]))

In [107]:
print('Shapiro Wilk for df_v2_expectations_before_laymen')
calculate_stats(df_v2_expectations_before_laymen)


Shapiro Wilk for df_v2_expectations_before_laymen


,Column,Shapiro-Wilk,p-value,Mean,Median
0,expectations.benefit-of-doubt,0.842306,2.602340e-09,5.289720,6.0
1,expectations.rely-on-ai-for-fake-news-detection,0.929945,2.743126e-05,4.644860,5.0
2,expectations.trust-ai,0.914216,3.647668e-06,4.560748,5.0
3,expectations.trust-new-tech,0.912570,2.988232e-06,4.663551,5.0


In [108]:
print('Shapiro Wilk for df_v2_expectations_after_laymen')
calculate_stats(df_v2_expectations_after_laymen)

Shapiro Wilk for df_v2_expectations_after_laymen


,Column,Shapiro-Wilk,p-value,Mean,Median
0,ai-system-evaluation.benefit-of-doubt,0.888312,6.300658e-09,5.253521,5.0
1,ai-system-evaluation.rely-on-ai-system,0.924295,7.458904e-07,4.591549,5.0
2,ai-system-evaluation.trust-new-tech,0.911751,1.241912e-07,4.894366,5.0


# Local Eval 
Für jedes News item einzeln besonders für vergleich zwischen TN FP pro Version und pro Gruppe

Auch 

Funktionen

In [110]:
def split_df_by_numbers(df):
    numbers = set()

    for col in df.columns:
        number = col.split('.')[1]
        numbers.add(number)

    # Dictionary zur Speicherung der DataFrames
    dfs = {}

    # Schleife über die Nummern
    for number in numbers:
        # Spalten mit der aktuellen Nummer extrahieren
        columns = [col for col in df.columns if col.split('.')[1] == number]

        # DataFrame mit den Spalten speichern
        df_number = df[columns].copy()
        dfs[f"df_{number}"] = df_number.dropna()

        # System-Evaluation DataFrames erstellen
        system_evaluation_columns = [col for col in columns if 'system-evaluation' in col]
        df_system_evaluation = df[system_evaluation_columns].copy()
        dfs[f"df_system_evaluation_{number}"] = df_system_evaluation.dropna()

        # Rating DataFrames erstellen
        rating_columns = [col for col in columns if 'rating' in col]
        df_rating = df[rating_columns].copy()
        dfs[f"df_rating_{number}"] = df_rating.dropna()

    return dfs

# Beispielaufruf#
#dfs = split_df_by_numbers(df_v1_news_items_laymen)


In [139]:
def process_ground_truth(df):
    # Liste aller Newsitems
    newsitems = ['newsitem.10', 'newsitem.11', 'newsitem.12', 'newsitem.13', 'newsitem.14', 'newsitem.7', 'newsitem.8', 'newsitem.9']
    df_dict = {}

    # Schleife über alle Newsitems
    for newsitem in newsitems:
        # Erstelle eine neue Spalte für "rating-before-xai_cat"
        df[f'{newsitem}.rating-before-xai_cat'] = pd.cut(df[f'{newsitem}.rating-before-xai'], 
                                                         bins=[0, 20, 40, 60, 80, 100], 
                                                         labels=['fake', 'rather fake', 'neutral', 'rather true', 'true'])
    
        # Erstelle eine neue Spalte für "rating-after-xai_cat"
        df[f'{newsitem}.rating-after-xai_cat'] = pd.cut(df[f'{newsitem}.rating-after-xai'], 
                                                        bins=[0, 20, 40, 60, 80, 100], 
                                                        labels=['fake', 'rather fake', 'neutral', 'rather true', 'true'])
    
        # Erstelle eine neue Spalte für "ground_truth"
        df[f'{newsitem}.ground_truth'] = (df[f'{newsitem}.rating-after-xai_cat'] != df[f'{newsitem}.rating-before-xai_cat']).astype(int)
    
        # Erstelle ein separates DataFrame für das aktuelle Newsitem
        df_newsitem = df[[f'{newsitem}.rating-before-xai_cat', f'{newsitem}.rating-after-xai_cat', f'{newsitem}.ground_truth']].copy()
        df_dict[newsitem] = df_newsitem.dropna()
        # Speichere das separate DataFrame als CSV-Datei
        # df_newsitem.to_csv(f'{newsitem}.csv', index=False)
    
    return df_dict


Berechnungen

In [135]:
df_v1_news_items_laymen = df_v1_laymen[[col for col in df_v1_laymen.columns if col.startswith('newsitem')]].copy()
df_v1_news_items_laymen

#columns = list(df_v1_news_items_laymen)
#pprint(columns)
#was passiert mit nan seperat machen?

,newsitem.10.control-question,newsitem.10.rating-after-xai,newsitem.10.rating-before-xai,newsitem.10.system-evaluation.classified-correctly,newsitem.10.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.10.system-evaluation.indications-useful,newsitem.10.system-evaluation.understand-what-system-does,newsitem.10.system-evaluation.xai-features-useful,newsitem.11.control-question,newsitem.11.rating-after-xai,...,newsitem.8.system-evaluation.understand-what-system-does,newsitem.8.system-evaluation.xai-features-useful,newsitem.9.control-question,newsitem.9.rating-after-xai,newsitem.9.rating-before-xai,newsitem.9.system-evaluation.classified-correctly,newsitem.9.system-evaluation.explanations-comprehensible-and-help-assess,newsitem.9.system-evaluation.indications-useful,newsitem.9.system-evaluation.understand-what-system-does,newsitem.9.system-evaluation.xai-features-useful
0,correct,90.0,90.0,7.0,7.0,6.0,7.0,6.0,correct,55,...,6,5,correct,38,38,5,5,5,6,5
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrong-4,64,...,6,7,wrong-1,19,84,7,4,4,6,5
2,correct,94.0,85.0,6.0,7.0,6.0,7.0,6.0,wrong-4,96,...,7,6,wrong-2,90,88,6,6,5,6,6
3,correct,29.0,55.0,5.0,4.0,2.0,4.0,4.0,correct,18,...,5,4,correct,8,24,7,5,6,4,6
4,correct,94.0,65.0,7.0,5.0,3.0,5.0,3.0,correct,21,...,7,4,correct,40,57,4,1,2,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,correct,75.0,62.0,6.0,5.0,5.0,4.0,5.0,correct,25,...,5,6,correct,16,39,6,5,5,5,5
138,correct,80.0,71.0,6.0,6.0,6.0,7.0,6.0,correct,8,...,7,4,correct,21,70,6,4,6,7,4
139,correct,87.0,40.0,6.0,6.0,7.0,6.0,6.0,correct,4,...,7,6,correct,13,32,6,7,7,7,7
140,correct,71.0,49.0,5.0,4.0,5.0,6.0,5.0,correct,21,...,6,5,correct,9,11,6,4,5,6,5


In [136]:
# Einzelne News items in dict packen
df_v1_news_items_grouped_laymen = split_df_by_numbers(df_v1_news_items_laymen)

# Beispiel: Ausgabe des DataFrames für Nummer '10'
df_number_10 = df_v1_news_items_grouped_laymen['df_10']
#print("DataFrame für Nummer 10:")
#df_number_10

# Beispiel: Ausgabe des System-Evaluation DataFrames für Nummer '10'
df_system_evaluation_10 = df_v1_news_items_grouped_laymen['df_system_evaluation_10']
#print("System-Evaluation DataFrame für Nummer 10:")
#df_system_evaluation_10

# Beispiel: Ausgabe des Rating DataFrames für Nummer '10'
df_rating_10 = df_v1_news_items_grouped_laymen['df_rating_10']
#print("Rating DataFrame für Nummer 10:")
#df_rating_10
#für alle weiteren auch 


DataFrame für Nummer 10:
System-Evaluation DataFrame für Nummer 10:
Rating DataFrame für Nummer 10:


In [132]:
calculate_alpha(df_system_evaluation_10)

(0.8782385868366606, array([0.843, 0.907]))

In [127]:
calculate_stats(df_system_evaluation_10)

,Column,Shapiro-Wilk,p-value,Mean,Median
0,newsitem.10.system-evaluation.classified-corre...,0.703904,1.652872e-15,6.156028,6.0
1,newsitem.10.system-evaluation.explanations-com...,0.810649,3.243650e-12,5.971631,6.0
2,newsitem.10.system-evaluation.indications-useful,0.815499,4.884098e-12,5.900709,6.0
3,newsitem.10.system-evaluation.understand-what-...,0.771664,1.531811e-13,6.177305,6.0
4,newsitem.10.system-evaluation.xai-features-useful,0.810510,3.206280e-12,5.957447,6.0


In [143]:
df_v1_processed_ground_truth_laymen = process_ground_truth(df_v1_news_items_laymen)

df_v1_newsitem_10_ground_truth_laymen = df_v1_processed_ground_truth_laymen['newsitem.10']
#df_v1_newsitem_11_ground_truth_laymen = df_v1_processed_ground_truth_laymen['newsitem.11']
# für alle news items 

#df_v1_newsitem_1_ground_truth_laymen
# Beispiel: Ausgabe des DataFrames für Nummer '10'


# hier auch schnitt für gesamte ground truth pro item? 

mean_value = df_v1_newsitem_10_ground_truth_laymen['newsitem.10.ground_truth'].mean()
median_value = df_v1_newsitem_10_ground_truth_laymen['newsitem.10.ground_truth'].median()

print("Mean der Ground Truth:", mean_value)
print("Median der Ground Truth:", median_value)

# tendenz zu 1 

df_v1_newsitem_10_ground_truth_laymen

Mean: 0.6714285714285714
Median: 1.0


,newsitem.10.rating-before-xai_cat,newsitem.10.rating-after-xai_cat,newsitem.10.ground_truth
0,true,true,0
2,true,true,0
3,neutral,rather fake,1
4,rather true,true,1
5,rather true,rather true,0
...,...,...,...
137,rather true,rather true,0
138,rather true,rather true,0
139,rather fake,true,1
140,neutral,rather true,1


# Globale Eval

wann nochmal 2. test für p value?? nach korrelationsanalyse?


In [170]:
def calculate_alpha_multiple(dfs):
    for i, df in enumerate(dfs):
        alpha = calculate_alpha(df)
        print(f"Alpha-Wert für DataFrame {i+1}: {alpha}")

def calculate_stats_multiple(dfs):
    for i, df in enumerate(dfs):
        stats = calculate_stats(df)
        print(f"Shapiro Wilk für DataFrame {i+1}: {stats}")



In [144]:
#Wie gloable performance? mean von lokal? 

In [161]:
# Trust 
column_names = ['ai-system-evaluation.system-supports-decision-making',
                'ai-system-evaluation.system-able-to-classify-news-articles',
                'ai-system-evaluation.system-can-correctly-classify-news-articles',
                'ai-system-evaluation.system-can-efficiently-classify-news-articles',
                'ai-system-evaluation.system-able-to-detect-fake-news']

df_v1_global_eval_trust_competence_laymen = df_v1_laymen[column_names].copy()
df_v2_global_eval_trust_competence_laymen = df_v2_laymen[column_names].copy()
df_v3_global_eval_trust_competence_laymen = df_v3_laymen[column_names].copy()

column_names = ['ai-system-evaluation.system-decides-neutral',
'ai-system-evaluation.system-decides-consistently',
'ai-system-evaluation.system-criteria-acceptable']

df_v1_global_eval_trust_integrity_laymen = df_v1_laymen[column_names].copy()
df_v2_global_eval_trust_integrity_laymen = df_v2_laymen[column_names].copy()
df_v3_global_eval_trust_integrity_laymen = df_v3_laymen[column_names].copy()


column_names = ['ai-system-evaluation.understand-why-system-provided-decision',
'ai-system-evaluation.understand-what-system-bases-decision-on',
'ai-system-evaluation.classification-comprehensible',
'ai-system-evaluation.easy-to-follow-what-system-does',
'news-dashboard-evaluation.satisfied-with-explanatory-quality'
]

df_v1_global_eval_trust_transparency_laymen = df_v1_laymen[column_names].copy()
df_v2_global_eval_trust_transparency_laymen = df_v2_laymen[column_names].copy()
df_v3_global_eval_trust_transparency_laymen = df_v3_laymen[column_names].copy()

column_names = ['ai-system-evaluation.know-what-will-happen-next-time']

df_v1_global_eval_trust_reuse_laymen = df_v1_laymen[column_names].copy()
df_v2_global_eval_trust_reuse_laymen = df_v2_laymen[column_names].copy()
df_v3_global_eval_trust_reuse_laymen = df_v3_laymen[column_names].copy()
# wie hier alpha?

column_names = ['ai-system-evaluation.tend-to-trust-system-even-without-knowledge',
'ai-system-evaluation.tendency-to-trust-system-high']

df_v1_global_eval_trust_overall_laymen = df_v1_laymen[column_names].copy()
df_v2_global_eval_trust_overall_laymen = df_v2_laymen[column_names].copy()
df_v3_global_eval_trust_overall_laymen = df_v3_laymen[column_names].copy()


In [171]:
# Beispiel
calculate_alpha_multiple([df_v1_global_eval_trust_competence_laymen, df_v2_global_eval_trust_competence_laymen, df_v3_global_eval_trust_competence_laymen])
calculate_stats_multiple([df_v1_global_eval_trust_competence_laymen, df_v2_global_eval_trust_competence_laymen, df_v3_global_eval_trust_competence_laymen])

Alpha-Wert für DataFrame 1: (0.9217064126799477, array([0.899, 0.94 ]))
Alpha-Wert für DataFrame 2: (0.9453434434491277, array([0.927, 0.96 ]))
Alpha-Wert für DataFrame 3: (0.9341996843302464, array([0.915, 0.95 ]))
Shapiro Wilk für DataFrame 1:                                               Column  Shapiro-Wilk  \
0  ai-system-evaluation.system-supports-decision-...      0.901492   
1  ai-system-evaluation.system-able-to-classify-n...      0.906623   
2  ai-system-evaluation.system-can-correctly-clas...      0.911353   
3  ai-system-evaluation.system-can-efficiently-cl...      0.906030   
4  ai-system-evaluation.system-able-to-detect-fak...      0.897601   

        p-value      Mean  Median  
0  3.194842e-08  5.098592     5.0  
1  6.230434e-08  5.133803     5.0  
2  1.176064e-07  5.161972     5.0  
3  5.761359e-08  5.154930     5.0  
4  1.952498e-08  5.084507     5.0  
Shapiro Wilk für DataFrame 2:                                               Column  Shapiro-Wilk  \
0  ai-system-eval

In [173]:
calculate_alpha_multiple([df_v1_global_eval_trust_integrity_laymen, df_v2_global_eval_trust_integrity_laymen, df_v3_global_eval_trust_integrity_laymen])
calculate_stats_multiple([df_v1_global_eval_trust_integrity_laymen, df_v2_global_eval_trust_integrity_laymen, df_v3_global_eval_trust_integrity_laymen])


Alpha-Wert für DataFrame 1: (0.8145390322695161, array([0.755, 0.862]))
Alpha-Wert für DataFrame 2: (0.6128214647798833, array([0.466, 0.725]))
Alpha-Wert für DataFrame 3: (0.765633919256737, array([0.69 , 0.825]))
Shapiro Wilk für DataFrame 1:                                              Column  Shapiro-Wilk  \
0       ai-system-evaluation.system-decides-neutral      0.946445   
1  ai-system-evaluation.system-decides-consistently      0.939179   
2   ai-system-evaluation.system-criteria-acceptable      0.925644   

        p-value      Mean  Median  
0  2.790034e-05  4.000000     4.0  
1  7.890811e-06  4.373239     5.0  
2  9.134072e-07  4.823944     5.0  
Shapiro Wilk für DataFrame 2:                                              Column  Shapiro-Wilk  \
0       ai-system-evaluation.system-decides-neutral      0.944832   
1  ai-system-evaluation.system-decides-consistently      0.899674   
2   ai-system-evaluation.system-criteria-acceptable      0.898516   

        p-value      Mean  

In [174]:
calculate_alpha_multiple([df_v1_global_eval_trust_transparency_laymen, df_v2_global_eval_trust_transparency_laymen, df_v3_global_eval_trust_transparency_laymen])
calculate_stats_multiple([df_v1_global_eval_trust_transparency_laymen, df_v2_global_eval_trust_transparency_laymen, df_v3_global_eval_trust_transparency_laymen])

Alpha-Wert für DataFrame 1: (0.9101290828504212, array([0.884, 0.932]))
Alpha-Wert für DataFrame 2: (0.9077867447017671, array([0.877, 0.933]))
Alpha-Wert für DataFrame 3: (0.8979708570117302, array([0.869, 0.922]))
Shapiro Wilk für DataFrame 1:                                               Column  Shapiro-Wilk  \
0  ai-system-evaluation.understand-why-system-pro...      0.897976   
1  ai-system-evaluation.understand-what-system-ba...      0.906225   
2  ai-system-evaluation.classification-comprehens...      0.918385   
3  ai-system-evaluation.easy-to-follow-what-syste...      0.912188   
4  news-dashboard-evaluation.satisfied-with-expla...      0.899140   

        p-value      Mean  Median  
0  2.046315e-08  5.112676     5.0  
1  5.911086e-08  5.042254     5.0  
2  3.142059e-07  5.176056     5.0  
3  1.318517e-07  5.112676     5.0  
4  2.369107e-08  5.338028     6.0  
Shapiro Wilk für DataFrame 2:                                               Column  Shapiro-Wilk  \
0  ai-system-eval

In [175]:
calculate_alpha_multiple([df_v1_global_eval_trust_overall_laymen, df_v2_global_eval_trust_overall_laymen, df_v3_global_eval_trust_overall_laymen])
calculate_stats_multiple([df_v1_global_eval_trust_overall_laymen, df_v2_global_eval_trust_overall_laymen, df_v3_global_eval_trust_overall_laymen])


Alpha-Wert für DataFrame 1: (0.9402528688242984, array([0.917, 0.957]))
Alpha-Wert für DataFrame 2: (0.9445458029896514, array([0.919, 0.962]))
Alpha-Wert für DataFrame 3: (0.93090157328568, array([0.904, 0.95 ]))
Shapiro Wilk für DataFrame 1:                                               Column  Shapiro-Wilk   p-value  \
0  ai-system-evaluation.tend-to-trust-system-even...      0.933819  0.000003   
1  ai-system-evaluation.tendency-to-trust-system-...      0.930683  0.000002   

       Mean  Median  
0  4.338028     5.0  
1  4.323944     5.0  
Shapiro Wilk für DataFrame 2:                                               Column  Shapiro-Wilk   p-value  \
0  ai-system-evaluation.tend-to-trust-system-even...      0.927910  0.000021   
1  ai-system-evaluation.tendency-to-trust-system-...      0.921207  0.000009   

       Mean  Median  
0  4.457944     5.0  
1  4.308411     5.0  
Shapiro Wilk für DataFrame 3:                                               Column  Shapiro-Wilk  \
0  ai-system

In [177]:
#Explanation Usefullness

column_names = ['news-dashboard-evaluation.increase-effectiveness-fake-news-detection',
'news-dashboard-evaluation.useful-assess-news-articles',
'news-dashboard-evaluation.help-detect-fake-news-faster',
'news-dashboard-evaluation.assist-decisions',
'news-dashboard-evaluation.useful-explanations-news',
'news-dashboard-evaluation.long-learning-time-not-worth'
]
df_v1_global_eval_explanation_usefulness_laymen = df_v1_laymen[column_names].copy()
df_v2_global_eval_explanation_usefulness_laymen = df_v2_laymen[column_names].copy()
df_v3_global_eval_explanation_usefulness_laymen = df_v3_laymen[column_names].copy()


# Reverse Coding der entsprechenden Frage
df_v1_global_eval_explanation_usefulness_laymen['news-dashboard-evaluation.long-learning-time-not-worth_reverse'] = 8 - df_v1_global_eval_explanation_usefulness_laymen['news-dashboard-evaluation.long-learning-time-not-worth']
df_v2_global_eval_explanation_usefulness_laymen['news-dashboard-evaluation.long-learning-time-not-worth_reverse'] = 8 - df_v2_global_eval_explanation_usefulness_laymen['news-dashboard-evaluation.long-learning-time-not-worth']
df_v3_global_eval_explanation_usefulness_laymen['news-dashboard-evaluation.long-learning-time-not-worth_reverse'] = 8 - df_v3_global_eval_explanation_usefulness_laymen['news-dashboard-evaluation.long-learning-time-not-worth']

df_v1_global_eval_explanation_usefulness_laymen.drop(['news-dashboard-evaluation.long-learning-time-not-worth'], axis=1, inplace=True)
df_v2_global_eval_explanation_usefulness_laymen.drop(['news-dashboard-evaluation.long-learning-time-not-worth'], axis=1, inplace=True)
df_v3_global_eval_explanation_usefulness_laymen.drop(['news-dashboard-evaluation.long-learning-time-not-worth'], axis=1, inplace=True)


In [181]:
calculate_alpha_multiple([df_v1_global_eval_explanation_usefulness_laymen, df_v2_global_eval_explanation_usefulness_laymen, df_v3_global_eval_explanation_usefulness_laymen])

calculate_stats_multiple([df_v1_global_eval_explanation_usefulness_laymen, df_v2_global_eval_explanation_usefulness_laymen, df_v3_global_eval_explanation_usefulness_laymen])

Alpha-Wert für DataFrame 1: (0.7876586169584889, array([0.729, 0.838]))
Alpha-Wert für DataFrame 2: (0.8557225994180409, array([0.809, 0.894]))
Alpha-Wert für DataFrame 3: (0.7641920615742603, array([0.698, 0.82 ]))
Shapiro Wilk für DataFrame 1:                                               Column  Shapiro-Wilk  \
0  news-dashboard-evaluation.increase-effectivene...      0.906789   
1  news-dashboard-evaluation.useful-assess-news-a...      0.895846   
2  news-dashboard-evaluation.help-detect-fake-new...      0.907439   
3         news-dashboard-evaluation.assist-decisions      0.883237   
4  news-dashboard-evaluation.useful-explanations-...      0.915654   
5  news-dashboard-evaluation.long-learning-time-n...      0.902587   

        p-value      Mean  Median  
0  6.368498e-08  5.345070     5.0  
1  1.569295e-08  5.422535     6.0  
2  6.942290e-08  5.133803     5.0  
3  3.480882e-09  5.450704     6.0  
4  2.133131e-07  5.140845     5.0  
5  3.677892e-08  4.746479     5.0  
Shapiro Wil

In [184]:
#Understandability
column_names = ['news-dashboard-evaluation.comprehensible-explanations-help-assess-news-articles',
'news-dashboard-evaluation.complicated-explainability-features',
'news-dashboard-evaluation.comprehensible-useful-metadata'
]
df_v1_global_eval_understandability_laymen = df_v1_laymen[column_names].copy()
df_v2_global_eval_understandability_laymen = df_v2_laymen[column_names].copy()
df_v3_global_eval_understandability_laymen = df_v3_laymen[column_names].copy()


# Reverse Coding der entsprechenden Frage
df_v1_global_eval_understandability_laymen['news-dashboard-evaluation.complicated-explainability-features_reverse'] = 8 - df_v1_global_eval_understandability_laymen['news-dashboard-evaluation.complicated-explainability-features']
df_v2_global_eval_understandability_laymen['news-dashboard-evaluation.complicated-explainability-features_reverse'] = 8 - df_v2_global_eval_understandability_laymen['news-dashboard-evaluation.complicated-explainability-features']
df_v3_global_eval_understandability_laymen['news-dashboard-evaluation.complicated-explainability-features_reverse'] = 8 - df_v3_global_eval_understandability_laymen['news-dashboard-evaluation.complicated-explainability-features']

df_v1_global_eval_understandability_laymen.drop(['news-dashboard-evaluation.complicated-explainability-features'], axis=1, inplace=True)
df_v2_global_eval_understandability_laymen.drop(['news-dashboard-evaluation.complicated-explainability-features'], axis=1, inplace=True)
df_v3_global_eval_understandability_laymen.drop(['news-dashboard-evaluation.complicated-explainability-features'], axis=1, inplace=True)


In [185]:
calculate_alpha_multiple([df_v1_global_eval_understandability_laymen,df_v2_global_eval_understandability_laymen, df_v3_global_eval_understandability_laymen])

Alpha-Wert für DataFrame 1: (0.29483585346052416, array([0.067, 0.474]))
Alpha-Wert für DataFrame 2: (0.5123143981005707, array([0.327, 0.653]))
Alpha-Wert für DataFrame 3: (0.1813585844400753, array([-0.084,  0.39 ]))


In [158]:
#pg.cronbach_alpha(data=df_v1_global_eval_understandability)
item, alpha = find_highest_alpha(df_v1_global_eval_understandability_laymen)

print("Highest Cronbach's Alpha:", alpha)
print("Item to remove:", item)
df_v1_global_eval_understandability_laymen.drop(['news-dashboard-evaluation.complicated-explainability-features_reverse'], axis=1, inplace=True)

Highest Cronbach's Alpha: (0.7529511782573666, array([0.656, 0.823]))
Item to remove: news-dashboard-evaluation.complicated-explainability-features_reverse


In [187]:
#pg.cronbach_alpha(data=df_v1_global_eval_understandability)
item, alpha = find_highest_alpha(df_v2_global_eval_understandability_laymen)

print("Highest Cronbach's Alpha:", alpha)
print("Item to remove:", item)
df_v2_global_eval_understandability_laymen.drop(['news-dashboard-evaluation.complicated-explainability-features_reverse'], axis=1, inplace=True)

Highest Cronbach's Alpha: (0.773781494411518, array([0.668, 0.846]))
Item to remove: news-dashboard-evaluation.complicated-explainability-features_reverse


In [189]:
#pg.cronbach_alpha(data=df_v1_global_eval_understandability)
item, alpha = find_highest_alpha(df_v3_global_eval_understandability_laymen)

print("Highest Cronbach's Alpha:", alpha)
print("Item to remove:", item)
df_v3_global_eval_understandability_laymen.drop(['news-dashboard-evaluation.complicated-explainability-features_reverse'], axis=1, inplace=True)

Highest Cronbach's Alpha: (0.7381711104836433, array([0.635, 0.812]))
Item to remove: news-dashboard-evaluation.complicated-explainability-features_reverse


In [190]:
calculate_alpha_multiple([df_v1_global_eval_understandability_laymen,df_v2_global_eval_understandability_laymen, df_v3_global_eval_understandability_laymen])

Alpha-Wert für DataFrame 1: (0.7529511782573666, array([0.656, 0.823]))
Alpha-Wert für DataFrame 2: (0.773781494411518, array([0.668, 0.846]))
Alpha-Wert für DataFrame 3: (0.7381711104836433, array([0.635, 0.812]))


In [191]:
calculate_stats_multiple([df_v1_global_eval_understandability_laymen,df_v2_global_eval_understandability_laymen, df_v3_global_eval_understandability_laymen])

Shapiro Wilk für DataFrame 1:                                               Column  Shapiro-Wilk  \
0  news-dashboard-evaluation.comprehensible-expla...      0.904033   
1  news-dashboard-evaluation.comprehensible-usefu...      0.886173   

        p-value      Mean  Median  
0  4.435553e-08  5.246479     5.0  
1  4.896713e-09  5.380282     6.0  
Shapiro Wilk für DataFrame 2:                                               Column  Shapiro-Wilk  \
0  news-dashboard-evaluation.comprehensible-expla...      0.868748   
1  news-dashboard-evaluation.comprehensible-usefu...      0.863374   

        p-value      Mean  Median  
0  2.788894e-08  5.327103     6.0  
1  1.683670e-08  5.411215     6.0  
Shapiro Wilk für DataFrame 3:                                               Column  Shapiro-Wilk  \
0  news-dashboard-evaluation.comprehensible-expla...      0.848841   
1  news-dashboard-evaluation.comprehensible-usefu...      0.868898   

        p-value      Mean  Median  
0  1.012184e-10  5.836879